In [1]:
import pandas as pd
import numpy as np

import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as goz

import plotly.figure_factory as ff

import warnings
warnings.filterwarnings('ignore')

py.init_notebook_mode(connected=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

Error importing optional module geopandas
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/_plotly_utils/optional_imports.py", line 30, in get_module
    return import_module(name)
  File "/opt/anaconda3/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/opt/anaconda3/lib/python3.9/site-packages/geopandas/__init__.py", line 1, in <module>
    from geopandas._config import options  # noqa
  File "/opt/anaconda3/lib/python3.9/site-packages/g

In [2]:
df=pd.read_csv("ceo_data_pay_merged_r3000.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [3]:
df.head()

,ticker,company_name,median_worker_pay,pay_ratio,ceo_name,salary,industry
0,ADV,"Advantage Solutions, Inc.","$14,328",721:1,Tanya Domier,"$10,330,444",Communication%20Services
1,AMC,"AMC Entertainment Holdings, Inc.","$9,386","2,015:1",Adam Aron,"$18,909,546",Communication%20Services
2,AMCX,AMC Networks Inc.,"$64,652",185:1,Matthew Blank,"$6,967,220",Communication%20Services
3,ATEX,Anterix Inc.,"$300,000",24:1,Robert Schwartz,"$6,532,403",Communication%20Services
4,ATNI,"ATN International, Inc.","$54,116",62:1,Michael Prior,"$3,379,851",Communication%20Services


### Cleaning Data

In [4]:
df["industry"] = df["industry"].str.replace(r'[^a-zA-Z_]', ' ', regex=True)
df["industry"] = df["industry"].replace(r'\s+', ' ', regex=True)

df["median_worker_pay"] = df["median_worker_pay"].str.replace(r"[$,]", "", regex=True)
df["median_worker_pay"] = pd.to_numeric(df["median_worker_pay"], errors="coerce")

df["salary"] = df["salary"].str.replace(r"[$,]", "", regex=True)
df["salary"] = pd.to_numeric(df["salary"], errors="coerce")

df["pay_ratio"]=df["pay_ratio"].str.split(':1').str[0]
df["pay_ratio"].replace(',','', regex=True, inplace=True)
df["pay_ratio"] = pd.to_numeric(df["pay_ratio"], errors="coerce")

In [5]:
df.sort_values(['salary'],ascending=False).head()

,ticker,company_name,median_worker_pay,pay_ratio,ceo_name,salary,industry
79,TTD,"The Trade Desk, Inc.",194939,4283,Jeff Green,834959367,Communication Services
172,EXPE,"Expedia Group, Inc.",102270,2897,Peter Kern,296247749,Consumer Discretionary
21,DISCA,"Warner Bros. Discovery, Inc.",82964,2972,David Zaslav,246573481,Communication Services
96,AMZN,"Amazon.com, Inc.",32855,6474,Andrew Jassy,212701169,Consumer Discretionary
1697,INTC,Intel Corporation,104400,1711,Patrick Gelsinger,178590400,Information Technology


### Feature Engineering

In [ ]:
import yfinance as yf
pd.set_option('display.max_rows', None)

industry=[]
sector=[]
fullTimeEmployees=[]
longBusinessSummary=[]
totalRevenue=[]
grossProfits=[]
freeCashflow=[]
operatingCashflow=[]
for i,ticker_i in enumerate(df["ticker"]):
#     try:
    ticker=yf.Ticker(str(ticker_i))
    print(i,ticker)

    try:
        industry.append(ticker.info["industry"])
    except:
#         print("industry-Issue",ticker)
        industry.append("Issue")

    try:
        sector.append(ticker.info["sector"])
    except:
#         print("sector-Issue",ticker)
        sector.append("Issue")

    try:
        fullTimeEmployees.append(ticker.info["fullTimeEmployees"])
    except:
#         print("fullTimeEmployees-Issue",ticker)
        fullTimeEmployees.append(np.nan)

    try:
        longBusinessSummary.append(ticker.info["longBusinessSummary"])
    except:
#         print("longBusinessSummary-Issue",ticker)
        longBusinessSummary.append("Issue")

    try:
        totalRevenue.append(ticker.info["totalRevenue"])
    except:
#         print("totalRevenue-Issue",ticker)
        totalRevenue.append(np.nan)

    try:
        grossProfits.append(ticker.info["grossProfits"])
    except:
#         print("grossProfits-Issue",ticker)
        grossProfits.append(np.nan)

    try:
        freeCashflow.append(ticker.info["freeCashflow"])
    except:
#         print("freeCashflow-Issue",ticker)
        freeCashflow.append(np.nan)

    try:
        operatingCashflow.append(ticker.info["operatingCashflow"])
    except:
#         print("operatingCashflow-Issue",ticker)
        operatingCashflow.append(np.nan)
#     except:
#         print("Issue",ticker)
#         industry.append("Issue")
#         sector.append("Issue")
#         fullTimeEmployees.append("Issue")
#         longBusinessSummary.append("Issue")
#         totalRevenue.append("Issue")
#         grossProfits.append("Issue")
#         freeCashflow.append("Issue")
#         operatingCashflow.append("Issue")

0 yfinance.Ticker object <ADV>
1 yfinance.Ticker object <AMC>
2 yfinance.Ticker object <AMCX>
3 yfinance.Ticker object <ATEX>
4 yfinance.Ticker object <ATNI>
5 yfinance.Ticker object <ATUS>
6 yfinance.Ticker object <ATVI>
7 yfinance.Ticker object <AUD>
8 yfinance.Ticker object <BAND>
9 yfinance.Ticker object <CABO>
10 yfinance.Ticker object <CARG>
11 yfinance.Ticker object <CARS>
12 yfinance.Ticker object <CCO>
13 yfinance.Ticker object <CCOI>
14 yfinance.Ticker object <CDLX>
15 yfinance.Ticker object <CHTR>
16 yfinance.Ticker object <CMCSA>
17 yfinance.Ticker object <CNK>
18 yfinance.Ticker object <CNSL>
19 yfinance.Ticker object <DHX>
20 yfinance.Ticker object <DIS>
21 yfinance.Ticker object <DISCA>
22 yfinance.Ticker object <DISH>
23 yfinance.Ticker object <EA>
24 yfinance.Ticker object <EB>
25 yfinance.Ticker object <EVC>
26 yfinance.Ticker object <EVER>
27 yfinance.Ticker object <FOXA>
28 yfinance.Ticker object <FWONA>
29 yfinance.Ticker object <FYBR>
30 yfinance.Ticker object <GC

248 yfinance.Ticker object <LZB>
249 yfinance.Ticker object <M>
250 yfinance.Ticker object <MAR>
251 yfinance.Ticker object <MAT>
252 yfinance.Ticker object <MBUU>
253 yfinance.Ticker object <MCD>
254 yfinance.Ticker object <MCFT>
255 yfinance.Ticker object <MCRI>
256 yfinance.Ticker object <MGM>
257 yfinance.Ticker object <MHK>
258 yfinance.Ticker object <MHO>
259 yfinance.Ticker object <MNRO>
260 yfinance.Ticker object <MOD>
261 yfinance.Ticker object <MOV>
262 yfinance.Ticker object <MPAA>
263 yfinance.Ticker object <MPX>
264 yfinance.Ticker object <MTH>
265 yfinance.Ticker object <MTN>
266 yfinance.Ticker object <MUSA>
267 yfinance.Ticker object <NCLH>
268 yfinance.Ticker object <NDLS>
269 yfinance.Ticker object <NKE>
270 yfinance.Ticker object <NVR>
271 yfinance.Ticker object <NWL>
272 yfinance.Ticker object <ODP>
273 yfinance.Ticker object <OLLI>
274 yfinance.Ticker object <ORLY>
275 yfinance.Ticker object <OSTK>
276 yfinance.Ticker object <OXM>
277 yfinance.Ticker object <PAG>
2

494 yfinance.Ticker object <DINO>
495 yfinance.Ticker object <DK>
496 yfinance.Ticker object <DO>
497 yfinance.Ticker object <DRQ>
498 yfinance.Ticker object <DVN>
499 yfinance.Ticker object <EFR>
500 yfinance.Ticker object <EGY>
501 yfinance.Ticker object <EOG>
502 yfinance.Ticker object <EQT>
503 yfinance.Ticker object <ESTE>
504 yfinance.Ticker object <ETRN>
505 yfinance.Ticker object <EVA>
506 yfinance.Ticker object <FANG>
507 yfinance.Ticker object <GEVO>
508 yfinance.Ticker object <GPOR>
509 yfinance.Ticker object <GPRE>
510 yfinance.Ticker object <HAL>
511 yfinance.Ticker object <HES>
512 yfinance.Ticker object <HLX>
513 yfinance.Ticker object <HP>
514 yfinance.Ticker object <INSW>
515 yfinance.Ticker object <INT>
516 yfinance.Ticker object <KMI>
517 yfinance.Ticker object <KOS>
518 yfinance.Ticker object <LBRT>
519 yfinance.Ticker object <LNG>
520 yfinance.Ticker object <LPG>
521 yfinance.Ticker object <LPI>
522 yfinance.Ticker object <MGY>
523 yfinance.Ticker object <MNRL>
524

739 yfinance.Ticker object <HBNC>
740 yfinance.Ticker object <HCI>
741 yfinance.Ticker object <HFWA>
742 yfinance.Ticker object <HIFS>
743 yfinance.Ticker object <HIG>
744 yfinance.Ticker object <HLI>
745 yfinance.Ticker object <HLNE>
746 yfinance.Ticker object <HMN>
747 yfinance.Ticker object <HMST>
748 yfinance.Ticker object <HOMB>
749 yfinance.Ticker object <HOPE>
750 yfinance.Ticker object <HTBI>
751 yfinance.Ticker object <HTBK>
752 yfinance.Ticker object <HTH>
753 yfinance.Ticker object <HTLF>
754 yfinance.Ticker object <HWC>
755 yfinance.Ticker object <IBCP>
756 yfinance.Ticker object <IBKR>
757 yfinance.Ticker object <IBTX>
758 yfinance.Ticker object <ICE>
759 yfinance.Ticker object <INBK>
760 yfinance.Ticker object <INDB>
761 yfinance.Ticker object <IVZ>
762 yfinance.Ticker object <JEF>
763 yfinance.Ticker object <JHG>
764 yfinance.Ticker object <JPM>
765 yfinance.Ticker object <JRVR>
766 yfinance.Ticker object <KEY>
767 yfinance.Ticker object <KMPR>
768 yfinance.Ticker object

984 yfinance.Ticker object <AVTR>
985 yfinance.Ticker object <AVXL>
986 yfinance.Ticker object <AXGN>
987 yfinance.Ticker object <AXSM>
988 yfinance.Ticker object <AZTA>
989 yfinance.Ticker object <BAX>
990 yfinance.Ticker object <BBIO>
991 yfinance.Ticker object <BCRX>
992 yfinance.Ticker object <BDX>
993 yfinance.Ticker object <BIIB>
994 yfinance.Ticker object <BIO>
995 yfinance.Ticker object <BKD>
996 yfinance.Ticker object <BLFS>
997 yfinance.Ticker object <BMRN>
998 yfinance.Ticker object <BMY>
999 yfinance.Ticker object <BPMC>
1000 yfinance.Ticker object <BRKR>
1001 yfinance.Ticker object <BSX>
1002 yfinance.Ticker object <CAH>
1003 yfinance.Ticker object <CANO>
1004 yfinance.Ticker object <CARA>
1005 yfinance.Ticker object <CCRN>
1006 yfinance.Ticker object <CDMO>
1007 yfinance.Ticker object <CDXS>
1008 yfinance.Ticker object <CERS>
1009 yfinance.Ticker object <CHE>
1010 yfinance.Ticker object <CHRS>
1011 yfinance.Ticker object <CI>
1012 yfinance.Ticker object <CLDX>
1013 yfinan

1222 yfinance.Ticker object <VIVO>
1223 yfinance.Ticker object <VNDA>
1224 yfinance.Ticker object <VREX>
1225 yfinance.Ticker object <VRTX>
1226 yfinance.Ticker object <VTRS>
1227 yfinance.Ticker object <VXRT>
1228 yfinance.Ticker object <WAT>
1229 yfinance.Ticker object <WST>
1230 yfinance.Ticker object <XNCR>
1231 yfinance.Ticker object <XRAY>
1232 yfinance.Ticker object <YMAB>
1233 yfinance.Ticker object <ZBH>
1234 yfinance.Ticker object <ZTS>
1235 yfinance.Ticker object <AAL>
1236 yfinance.Ticker object <AAON>
1237 yfinance.Ticker object <AAWW>
1238 yfinance.Ticker object <ABM>
1239 yfinance.Ticker object <ACA>
1240 yfinance.Ticker object <ACCO>
1241 yfinance.Ticker object <ACM>
1242 yfinance.Ticker object <AGCO>
1243 yfinance.Ticker object <AGX>
1244 yfinance.Ticker object <AIMC>
1245 yfinance.Ticker object <AIN>
1246 yfinance.Ticker object <AIR>
1247 yfinance.Ticker object <AIT>
1248 yfinance.Ticker object <AJRD>
1249 yfinance.Ticker object <AL>
1250 yfinance.Ticker object <ALG>


1461 yfinance.Ticker object <OSK>
1462 yfinance.Ticker object <OTIS>
1463 yfinance.Ticker object <PBI>
1464 yfinance.Ticker object <PGTI>
1465 yfinance.Ticker object <PH>
1466 yfinance.Ticker object <PKE>
1467 yfinance.Ticker object <PLOW>
1468 yfinance.Ticker object <PLPC>
1469 yfinance.Ticker object <PLUG>
1470 yfinance.Ticker object <PNR>
1471 yfinance.Ticker object <POWL>
1472 yfinance.Ticker object <PRIM>
1473 yfinance.Ticker object <PRLB>
1474 yfinance.Ticker object <PSN>
1475 yfinance.Ticker object <PWR>
1476 yfinance.Ticker object <R>
1477 yfinance.Ticker object <REVG>
1478 yfinance.Ticker object <REZI>
1479 yfinance.Ticker object <RGP>
1480 yfinance.Ticker object <RHI>
1481 yfinance.Ticker object <RLGT>
1482 yfinance.Ticker object <ROAD>
1483 yfinance.Ticker object <ROCK>
1484 yfinance.Ticker object <ROK>
1485 yfinance.Ticker object <ROL>
1486 yfinance.Ticker object <ROLL>
1487 yfinance.Ticker object <RRX>
1488 yfinance.Ticker object <RSG>
1489 yfinance.Ticker object <RTX>
149

1700 yfinance.Ticker object <IT>
1701 yfinance.Ticker object <ITRI>
1702 yfinance.Ticker object <JBL>
1703 yfinance.Ticker object <JKHY>
1704 yfinance.Ticker object <JNPR>
1705 yfinance.Ticker object <KE>
1706 yfinance.Ticker object <KEYS>
1707 yfinance.Ticker object <KLAC>
1708 yfinance.Ticker object <KLIC>
1709 yfinance.Ticker object <KN>
1710 yfinance.Ticker object <LASR>
1711 yfinance.Ticker object <LFUS>
1712 yfinance.Ticker object <LITE>
1713 yfinance.Ticker object <LLNW>
1714 yfinance.Ticker object <LPSN>
1715 yfinance.Ticker object <LRCX>
1716 yfinance.Ticker object <LSCC>
1717 yfinance.Ticker object <LWLG>
1718 yfinance.Ticker object <MA>
1719 yfinance.Ticker object <MANH>
1720 yfinance.Ticker object <MAPS>
1721 yfinance.Ticker object <MCHP>
1722 yfinance.Ticker object <MDB>
1723 yfinance.Ticker object <MEI>
1724 yfinance.Ticker object <MGI>
1725 yfinance.Ticker object <MITK>
1726 yfinance.Ticker object <MKSI>
1727 yfinance.Ticker object <MMS>
1728 yfinance.Ticker object <MNTV

In [ ]:
df["industry"]=industry
df["sector"]=sector
df["fullTimeEmployees"]=fullTimeEmployees
df["longBusinessSummary"]=longBusinessSummary
df["totalRevenue"]=totalRevenue
df["grossProfits"]=grossProfits
df["freeCashflow"]=freeCashflow
df["operatingCashflow"]=operatingCashflow

In [ ]:
df.dtypes

In [ ]:
df_before_na = df.copy(deep=True)

In [ ]:
df=df.fillna(0)
# df=df.dropna()

In [ ]:
df.head(10)

In [ ]:
df.to_csv("Russell3000.csv",index=False)

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
import plotly.figure_factory as ff

pd.set_option('display.float_format', lambda x: '%.2f' % x)

df=pd.read_csv("Russell3000.csv")

### Univariate Analysis

In [ ]:
# num_cols=df.select_dtypes(include=[np.number])

num_cols = list(df.select_dtypes(include = np.number).columns)

In [ ]:
colorscale = [[0, '#1f77b4'],[.5, '#aec7e8'],[1, '#ff7f0e']]
fig = ff.create_table(round(df[num_cols].describe().reset_index(), 2), font_colors = ['white'], colorscale = colorscale)

for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 9
    
fig.update_layout(
    title_text = 'Descriptive Statistics on the numerical columns',
    margin = {'t':50},
    template= "plotly_dark"
)
    
fig.show()

In [ ]:
# categorized_cols=df.select_dtypes(exclude=[object])

categorized_cols = list(df.select_dtypes(include = 'object').columns)

In [ ]:
colorscale = [[0, '#1f77b4'],[.5, '#aec7e8'],[1, '#ff7f0e']]
fig = ff.create_table(df[categorized_cols].describe().reset_index(), font_colors = ['white'], colorscale = colorscale)

for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 9
    
fig.update_layout(
    title_text = 'Statistics of the categorical columns',
    margin = {'t':50},
    template= "plotly_dark"
)
    
fig.show()

In [ ]:
num_cols

In [ ]:
def plot_histogram(col_name):
    series = df[col_name]
    # remove zero values items
    series = series[ series != 0 ]
    smin,smax = series.min(),series.max()
    # remove outliers for +- three standard deviations.
    series = series[ ~( ( series - series.mean() ).abs() > 3 * series.std() ) ]
    percentiles = [ np.percentile(series,n) for n in (2.5,50,97.5) ]
    
    trace0 = go.Histogram( x = series,
                            histfunc = 'avg', 
                            histnorm = 'probability density',
                            opacity=.75,
                            marker = {'color':'#EB89B5'})
    data_ = go.Data( [trace0] )
    
    shapes = [{ 'line': { 'color': '#0099FF', 'dash':'solid', 'width':2 },
                'type':'line',
                'x0':percentiles[0], 'x1':percentiles[0], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' },
    { 'line': { 'color': '#00999F', 'dash':'solid', 'width':1 },
                'type':'line',
                'x0':percentiles[1], 'x1':percentiles[1], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' },
    
              { 'line': { 'color': '#0099FF', 'dash':'solid', 'width':2 },
                'type':'line',
                'x0':percentiles[2], 'x1':percentiles[2], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' } 
             ]
    
    annotations = [ {'x': percentiles[0], 'xref':'x','xanchor':'right',
                     'y': .3, 'yref':'paper', 
                     'text':'2.5%', 'font':{'size':16},
                     'showarrow':False},
                   
                    {'x': percentiles[1], 'xref':'x','xanchor':'center',
                     'y': .2, 'yref':'paper', 
                     'text':'95%<br>median = {0:,.2f}<br>mean = {1:,.2f}<br>min = {2:,}<br>max = {3:,}'
                         .format(percentiles[1],series.mean(),smin,smax), 
                     'showarrow':False,
                     'font':{'size':20} },
                   
    {'x': percentiles[2], 'xref':'x','xanchor':'left',
                     'y': .3, 'yref':'paper', 
                     'text':'2.5%','font':{'size':16}, 
                     'showarrow':False},
                   
                    {'x': .5, 'xref':'paper','xanchor':'center',
                     'y': 1.1, 'yref':'paper','yanchor':'middle', 
                     'text':'Outliers above or below three standard deviations are excluded from the graph, mean and median calculations.',
                     'font':{'size':15,'color':'mistyrose'}, 
                     'showarrow':False} 
                  ]
    
    layout = go.Layout( title = col_name.replace('_',' ').capitalize(),
                        titlefont = {'size':50},
                        yaxis = {'title':'Probability/Density'},
                        xaxis = {'title':col_name, 'type':'linear'},
                        shapes = shapes,
                         annotations = annotations
                         )
    figure = go.Figure(data = data_, layout = layout)
    py.iplot(figure)
    
def plot_histogram_without_outliers(col_name):
    series = df[col_name]
    # remove zero values items
    series = series[ series != 0 ]
    smin,smax = series.min(),series.max()
    # remove outliers for +- three standard deviations.
#     series = series[ ~( ( series - series.mean() ).abs() > 3 * series.std() ) ]
    percentiles = [ np.percentile(series,n) for n in (2.5,50,97.5) ]
    
    trace0 = go.Histogram( x = series,
                            histfunc = 'avg', 
                            histnorm = 'probability density',
                            opacity=.75,
                            marker = {'color':'#EB89B5'})
    data_ = go.Data( [trace0] )
    
    shapes = [{ 'line': { 'color': '#0099FF', 'dash':'solid', 'width':2 },
                'type':'line',
                'x0':percentiles[0], 'x1':percentiles[0], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' },
    { 'line': { 'color': '#00999F', 'dash':'solid', 'width':1 },
                'type':'line',
                'x0':percentiles[1], 'x1':percentiles[1], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' },
    
              { 'line': { 'color': '#0099FF', 'dash':'solid', 'width':2 },
                'type':'line',
                'x0':percentiles[2], 'x1':percentiles[2], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' } 
             ]
    
    annotations = [ {'x': percentiles[0], 'xref':'x','xanchor':'right',
                     'y': .3, 'yref':'paper', 
                     'text':'2.5%', 'font':{'size':16},
                     'showarrow':False},
                   
                    {'x': percentiles[1], 'xref':'x','xanchor':'center',
                     'y': .2, 'yref':'paper', 
                     'text':'95%<br>median = {0:,.2f}<br>mean = {1:,.2f}<br>min = {2:,}<br>max = {3:,}'
                         .format(percentiles[1],series.mean(),smin,smax), 
                     'showarrow':False,
                     'font':{'size':20} },
                   
    {'x': percentiles[2], 'xref':'x','xanchor':'left',
                     'y': .3, 'yref':'paper', 
                     'text':'2.5%','font':{'size':16}, 
                     'showarrow':False},
                   
                    {'x': .5, 'xref':'paper','xanchor':'center',
                     'y': 1.1, 'yref':'paper','yanchor':'middle', 
                     'text':'Outliers above or below three standard deviations are excluded from the graph, mean and median calculations.',
                     'font':{'size':15,'color':'mistyrose'}, 
                     'showarrow':False} 
                  ]
    
    layout = go.Layout( title = col_name.replace('_',' ').capitalize(),
                        titlefont = {'size':50},
                        yaxis = {'title':'Probability/Density'},
                        xaxis = {'title':col_name, 'type':'linear'},
                        shapes = shapes,
                         annotations = annotations
                         )
    figure = go.Figure(data = data_, layout = layout)
    py.iplot(figure)
    
def plot_value_counts(col_name, table=False, bar=False):
    N=10
    values_count = pd.DataFrame(df[col_name].value_counts())
    values_count.columns = ['count']
    # convert the index column into a regular column.
    values_count[col_name] = [ str(i) for i in values_count.index ]
    # add a column with the percentage of each data point to the sum of all data points.
    values_count['percent'] = values_count['count'].div(values_count['count'].sum()).multiply(100).round(2)
    # change the order of the columns.
    values_count = values_count.reindex([col_name,'count','percent'],axis=1)
    values_count.reset_index(drop=True,inplace=True)
    values_count=values_count.head(N)
    if bar:
        # add a font size for annotations0 which is relevant to the length of the data points.
        font_size = int(abs(20 - (.25 * len(values_count[col_name]))))
        
        trace0 = go.Bar(x=values_count[col_name], y=values_count['count'])
        data_ = go.Data([trace0])
        
        annotations0 = [dict(x=xi,
                             y=yi, 
                             showarrow=False,
                             font={'size':font_size},
                             text="{:,}".format(yi),
                             xanchor='center',
                             yanchor='bottom')
                        for xi,yi,_ in values_count.values]
        
        annotations1 = [dict(x=xi,
                             y=yi/2,
                             showarrow=False,
                             text="{}%".format(pi),
                             xanchor='center',
                             yanchor='middle',
                             font={'color':'yellow'})
                        for xi,yi,pi in values_count.values if pi > 10]
        
        annotations = annotations0 + annotations1
        
        layout = go.Layout(title=col_name.replace('_',' ').capitalize(),
                           titlefont={'size':50},
                           yaxis={'title':'count'},
                           xaxis={'type':'category'},
                           annotations=annotations)
        figure = go.Figure(data=data_, layout=layout)
        py.iplot(figure)
    
    if table: 
        values_count['count'] = values_count['count'].apply(lambda d : "{:,}".format(d))
        table = ff.create_table(values_count,index_title="race")
        py.iplot(table)
    
    return values_count


In [ ]:
for col in num_cols :
    plot_histogram(col)
    
for col in num_cols :
    plot_histogram_without_outliers(col)
    

In [ ]:
categorized_cols

In [ ]:
# Remove specific columns from the categorized_cols list
categorized_cols = [col for col in categorized_cols if col not in ['ticker', 'company_name', 'ceo_name', 'longBusinessSummary']]


In [ ]:
df_cat=[]
for col in categorized_cols:
    values_count = plot_value_counts(col,1,1)
    df_cat.append(values_count)

### Bivariate Analysis

In [ ]:
import plotly.express as px
px.scatter(df, x="median_worker_pay", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")

In [ ]:
px.scatter(df, x="pay_ratio", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")

In [ ]:
px.scatter(df, x="totalRevenue", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")

In [ ]:
px.scatter(df, x="grossProfits", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")

In [ ]:
px.scatter(df, x="freeCashflow", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")

In [ ]:
px.scatter(df, x="operatingCashflow", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")